In [1]:
from pathlib import Path
import pandas as pd
import sys
ROOT = Path().resolve().parent
src_folder = ROOT / "src"
if str(src_folder) not in sys.path:
    sys.path.insert(0, str(src_folder))
from data_workflow.viz import create_bar, create_line, create_histogram, SaveFig
from data_workflow.utils import bootstrap_diff_means

In [2]:
DATA = ROOT / "data/processed/analytics_table.parquet"
# How many rows and columns 
df = pd.read_parquet(DATA)
df.shape


(5, 18)

In [3]:
audit = pd.DataFrame({"dtype": df.dtypes.astype(str),"missing_count": df.isna().sum(),"missing_percentage": df.isna().mean(),
}).sort_values("missing_percentage", ascending=False)
# missing values in each column
audit.head(5)

,dtype,missing_count,missing_percentage
date,object,1,0.2
year,float64,1,0.2
amount_w,Float64,1,0.2
hour,float64,1,0.2
dow,float64,1,0.2


In [4]:
revenue_by_country = (df.groupby("country", dropna=False)["amount"].sum().reset_index())

barchart = create_bar(revenue_by_country, x_col="country",y_col="amount",plot_title="Revenue by country (all time)",)

FIGS = ROOT / "reports/figures"
FIGS.mkdir(parents=True, exist_ok=True)
# highest total revenue
SaveFig(barchart, FIGS / "revenue_by_country.png")
barchart


In [5]:
#monthly trend

monthly_trend = (df.groupby("month", dropna=False)["amount"].sum().reset_index().sort_values("month"))
fig = create_line(monthly_trend,x_col="month",y_col="amount",plot_title="Monthly revenue trend",)
SaveFig(fig, FIGS / "monthly_revenue_trend.png")
fig


In [6]:
df["month"].nunique()


1

In [7]:
#distribution of order amounts winsorized
histogram = create_histogram(df,column="amount_w",bins=30,plot_title="Distribution of order amounts (winsorized)",)
SaveFig(histogram, FIGS / "amount_distribution.png")
histogram

In [8]:
refund_data = df.assign(is_refund=(df["status_clean"] == "refund").astype(int))
#statistically meaningful difference in refund rates between SA and AE?
sa_rates = refund_data.loc[refund_data["country"] == "SA", "is_refund"]
ae_rates = refund_data.loc[refund_data["country"] == "AE", "is_refund"]
analysis = bootstrap_diff_means(sa_rates, ae_rates, n_boot=2000, seed=0)
analysis

{'diff_mean': -1.0, 'ci_lower': -1.0, 'ci_upper': -1.0}

In [9]:
df.groupby("country")["amount"].sum()


country
AE      0.0
SA    145.5
Name: amount, dtype: Float64

### Interpretation

- Saudi Arabia generates higher total revenue compared to the UAE.
- Revenue changes across months, which may indicate possible seasonality.
- The distribution of order amounts is right-skewed; winsorization helps reduce the effect of extreme values.
- The bootstrap analysis estimates the difference in refund rates between Saudi Arabia and the UAE.
- The confidence interval helps determine whether the observed difference is statistically meaningful.


